# **Homework 2-1 Phoneme Classification**

<font color = "sky blue">Model architecture

1.   layer(7)
2.   dim(4096->2048->1024->512->256->128->->39)
3.   activation function (<font color = "red">ReLU</font>: 0.67812, RReLU:0.68872, ReLU6: 0.682, ELU)

</font>

<font color = "white">Training
1. batch size (17)
2. optimizer (Adam)
3. learning rate (0.00007; learning schedule?)
4. epoch (50)
5. val_ratio = 0.1

</font>

<font color = "sky blue">Tips
1. batch norm (1d)
2. drop out (0.4, 0.3, 0.3, 0.2, 0.2, 0.1)
3. regularization(weight_decay = 0.001)

</font>

<font color = "lime">status

</font>

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
'''
!gdown --id '1IUjpShq9Q8oSGpiUBd4HseZYsX5yC0sC' --output data.zip
!unzip data.zip
!ls
'''

"\n!gdown --id '1IUjpShq9Q8oSGpiUBd4HseZYsX5yC0sC' --output data.zip\n!unzip data.zip\n!ls\n"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [3]:
import numpy as np

print('Loading data ...')

data_root='./drive/MyDrive/ML2021/timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [5]:
VAL_RATIO = 0.1 #TODO

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [6]:
BATCH_SIZE = 17 #TODO

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [7]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

100

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [13]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self): #TODO
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 4096)
        self.layer2 = nn.Linear(4096, 2048)
        self.layer3 = nn.Linear(2048, 1024)
        self.layer4 = nn.Linear(1024, 512)
        self.layer5 = nn.Linear(512, 256)
        self.layer6 = nn.Linear(256, 128)
        #self.layer7 = nn.Linear(128, 64)
        self.out = nn.Linear(128, 39)

        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.2)
        self.dropout5 = nn.Dropout(0.2)
        self.dropout6 = nn.Dropout(0.1)
        #self.dropout7 = nn.Dropout(0.2)

        self.batchnorm1 = nn.BatchNorm1d(4096)
        self.batchnorm2 = nn.BatchNorm1d(2048)
        self.batchnorm3 = nn.BatchNorm1d(1024)
        self.batchnorm4 = nn.BatchNorm1d(512)
        self.batchnorm5 = nn.BatchNorm1d(256)
        self.batchnorm6 = nn.BatchNorm1d(128)
        #self.batchnorm7 = nn.BatchNorm1d(64)

        self.act_fn = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.act_fn(x)
        x = self.batchnorm1(x)
        x = self.dropout1(x)

        x = self.layer2(x)
        x = self.act_fn(x)
        x = self.batchnorm2(x)
        x = self.dropout2(x)

        x = self.layer3(x)
        x = self.act_fn(x)
        x = self.batchnorm3(x)
        x = self.dropout3(x)

        x = self.layer4(x)
        x = self.act_fn(x)
        x = self.batchnorm4(x)
        x = self.dropout4(x)

        x = self.layer5(x)
        x = self.act_fn(x)
        x = self.batchnorm5(x)
        x = self.dropout5(x)

        x = self.layer6(x)
        x = self.act_fn(x)
        x = self.batchnorm6(x)
        x = self.dropout6(x)

        x = self.out(x)
        
        return x

## Training

In [14]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [15]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [16]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters 
num_epoch = 60              # number of training epoch #TODO
learning_rate = 0.00007       # learning rate #TODO

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  #TODO

DEVICE: cuda


In [17]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch   !!!!!
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/060] Train Acc: 0.555980 Loss: 1.484888 | Val Acc: 0.681375 loss: 1.028471
saving model with acc 0.681
[002/060] Train Acc: 0.626198 Loss: 1.210318 | Val Acc: 0.701278 loss: 0.943090
saving model with acc 0.701
[003/060] Train Acc: 0.649261 Loss: 1.124153 | Val Acc: 0.712279 loss: 0.899453
saving model with acc 0.712
[004/060] Train Acc: 0.663978 Loss: 1.070934 | Val Acc: 0.721629 loss: 0.861974
saving model with acc 0.722
[005/060] Train Acc: 0.674360 Loss: 1.030606 | Val Acc: 0.725450 loss: 0.847756
saving model with acc 0.725
[006/060] Train Acc: 0.682910 Loss: 1.000674 | Val Acc: 0.730515 loss: 0.827797
saving model with acc 0.731
[007/060] Train Acc: 0.689685 Loss: 0.976258 | Val Acc: 0.732686 loss: 0.818313
saving model with acc 0.733
[008/060] Train Acc: 0.694884 Loss: 0.957011 | Val Acc: 0.731702 loss: 0.814115
[009/060] Train Acc: 0.700012 Loss: 0.938365 | Val Acc: 0.737882 loss: 0.799983
saving model with acc 0.738
[010/060] Train Acc: 0.704926 Loss: 0.921932 | Val Acc: 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [18]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [19]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [20]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))